In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os
import nfl_data_py as nfl

pd.set_option('display.max_columns', None)

In [2]:
# Check current directory
os.getcwd()

'C:\\Users\\imacd_0odruq3\\Documents\\git_repos\\nfl_prediction\\data_transformation'

In [3]:
# Use nfl_data_py package to get data

nfl_df = nfl.import_pbp_data([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [4]:
nfl_df.head(5)

play_id          game_id old_game_id home_team away_team season_type  week  \
0      1.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
1     36.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
2     51.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
3     72.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
4     93.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   

  posteam posteam_type defteam side_of_field  yardline_100   game_date  \
0    None         None    None          None           NaN  2014-09-07   
1     BUF         away     CHI           CHI          35.0  2014-09-07   
2     BUF         away     CHI           BUF          80.0  2014-09-07   
3     BUF         away     CHI           BUF          78.0  2014-09-07   
4     BUF         away     CHI           BUF          76.0  2014-09-07   

   quarter_seconds_remaining  half_seconds_remaining  game_seconds_remaining  \
0                      900.0                  1800.0                  3600.0   
1                      900.0                  1800.0                  3600.0   
2                      900.0                  1800.0                  3600.0   
3                      867.0                  1767.0                  3567.0   
4                      823.0                  1723.0                  3523.0   

  game_half  quarter_end  drive   sp  qtr  down  goal_to_go   time   yrdln  \
0     Half1          0.0    NaN  0.0  1.0   NaN         0.0  15:00  CHI 35   
1     Half1          0.0    1.0  0.0  1.0   NaN         0.0  15:00  CHI 35   
2     Half1          0.0    1.0  0.0  1.0   1.0         0.0  15:00  BUF 20   
3     Half1          0.0    1.0  0.0  1.0   2.0         0.0  14:27  BUF 22   
4     Half1          0.0    1.0  0.0  1.0   3.0         0.0  13:43  BUF 24   

   ydstogo  ydsnet                                               desc  \
0      0.0     NaN                                               GAME   
1      0.0     4.0  9-R.Gould kicks 65 yards from CHI 35 to end zo...   
2     10.0     4.0  (15:00) 28-C.Spiller left end to BUF 22 for 2 ...   
3      8.0     4.0  (14:27) (Shotgun) 28-C.Spiller left tackle to ...   
4      6.0     4.0  (13:43) (Shotgun) 3-E.Manuel pass incomplete s...   

  play_type  yards_gained  shotgun  no_huddle  qb_dropback  qb_kneel  \
0      None           NaN      0.0        0.0          NaN       0.0   
1   kickoff           0.0      0.0        0.0          0.0       0.0   
2       run           2.0      0.0        0.0          0.0       0.0   
3       run           2.0      1.0        0.0          0.0       0.0   
4      pass           0.0      1.0        0.0          1.0       0.0   

   qb_spike  qb_scramble pass_length pass_location  air_yards  \
0       0.0          0.0        None          None        NaN   
1       0.0          0.0        None          None        NaN   
2       0.0          0.0        None          None        NaN   
3       0.0          0.0        None          None        NaN   
4       0.0          0.0       short        middle       11.0   

   yards_after_catch run_location run_gap field_goal_result  kick_distance  \
0                NaN         None    None              None            NaN   
1                NaN         None    None              None            NaN   
2                NaN         left     end              None            NaN   
3                NaN         left  tackle              None            NaN   
4                NaN         None    None              None            NaN   

  extra_point_result two_point_conv_result  home_timeouts_remaining  \
0               None                  None                      3.0   
1               None                  None                      3.0   
2               None                  None                      3.0   
3               None                  None                      3.0   
4               None              

In [5]:
# Add column detailing if there was a qb designed run

rosters = nfl.import_rosters([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])
qb_roster = rosters[rosters.position == 'QB']
qb_roster = qb_roster[['position', 'player_name', 'player_id']].drop_duplicates()
qb_roster['player_id_string'] = qb_roster.apply(lambda x: str(x.player_id), axis=1)

def is_designed_qb_run(play, qb_roster):
    if play.rusher_id:
        if qb_roster.player_id_string.str.contains(str(play.rusher_id)).any():
            return 1 
    else: 
        return 0 
    
nfl_df['qb_designed_run'] = nfl_df.apply(lambda x: is_designed_qb_run(x, qb_roster), axis=1)

In [6]:
# Get important offensive NFL stats per game per team

# Get only offensive plays, take out qb kneels

offense = nfl_df[nfl_df.play_type.isin(['run', 'pass'])]
offense = offense[offense.qb_kneel == 0]
offense.head(5)

play_id          game_id old_game_id home_team away_team season_type  week  \
2     51.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
3     72.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
4     93.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
6    141.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
7    162.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   

  posteam posteam_type defteam side_of_field  yardline_100   game_date  \
2     BUF         away     CHI           BUF          80.0  2014-09-07   
3     BUF         away     CHI           BUF          78.0  2014-09-07   
4     BUF         away     CHI           BUF          76.0  2014-09-07   
6     CHI         home     BUF           CHI          66.0  2014-09-07   
7     CHI         home     BUF           CHI          64.0  2014-09-07   

   quarter_seconds_remaining  half_seconds_remaining  game_seconds_remaining  \
2                      900.0                  1800.0                  3600.0   
3                      867.0                  1767.0                  3567.0   
4                      823.0                  1723.0                  3523.0   
6                      810.0                  1710.0                  3510.0   
7                      768.0                  1668.0                  3468.0   

  game_half  quarter_end  drive   sp  qtr  down  goal_to_go   time   yrdln  \
2     Half1          0.0    1.0  0.0  1.0   1.0         0.0  15:00  BUF 20   
3     Half1          0.0    1.0  0.0  1.0   2.0         0.0  14:27  BUF 22   
4     Half1          0.0    1.0  0.0  1.0   3.0         0.0  13:43  BUF 24   
6     Half1          0.0    2.0  0.0  1.0   1.0         0.0  13:30  CHI 34   
7     Half1          0.0    2.0  0.0  1.0   2.0         0.0  12:48  CHI 36   

   ydstogo  ydsnet                                               desc  \
2     10.0     4.0  (15:00) 28-C.Spiller left end to BUF 22 for 2 ...   
3      8.0     4.0  (14:27) (Shotgun) 28-C.Spiller left tackle to ...   
4      6.0     4.0  (13:43) (Shotgun) 3-E.Manuel pass incomplete s...   
6     10.0    66.0  (13:30) (Shotgun) 22-M.Forte right guard to CH...   
7      8.0    66.0  (12:48) (Shotgun) 6-J.Cutler pass short right ...   

  play_type  yards_gained  shotgun  no_huddle  qb_dropback  qb_kneel  \
2       run           2.0      0.0        0.0          0.0       0.0   
3       run           2.0      1.0        0.0          0.0       0.0   
4      pass           0.0      1.0        0.0          1.0       0.0   
6       run           2.0      1.0        0.0          0.0       0.0   
7      pass           8.0      1.0        0.0          1.0       0.0   

   qb_spike  qb_scramble pass_length pass_location  air_yards  \
2       0.0          0.0        None          None        NaN   
3       0.0          0.0        None          None        NaN   
4       0.0          0.0       short        middle       11.0   
6       0.0          0.0        None          None        NaN   
7       0.0          0.0       short         right        4.0   

   yards_after_catch run_location run_gap field_goal_result  kick_distance  \
2                NaN         left     end              None            NaN   
3                NaN         left  tackle              None            NaN   
4                NaN         None    None              None            NaN   
6                NaN        right   guard              None            NaN   
7                4.0         None    None              None            NaN   

  extra_point_result two_point_conv_result  home_timeouts_remaining  \
2               None                  None                      3.0   
3               None                  None                      3.0   
4               None                  None                      3.0   
6               None                  None                      3.0   
7               None              

In [7]:
# Get rushing stats

# Total rushes, total rush yards, rushing epa

rushes = offense[offense.play_type == 'run']
rushing_grouped = rushes.groupby(by=['season', 'week', 'posteam'])
rush_df = rushing_grouped.count()['play_id'].rename('total_rushes').to_frame()
rush_df[['total_rush_yards', 'rushing_epa', 'rush_tds']] = rushing_grouped.sum()[['yards_gained', 'epa', 'rush_touchdown']]

rush_df.head(5)

total_rushes  total_rush_yards  rushing_epa  rush_tds
season week posteam                                                       
2014   1    ARI                26             110.0    -7.697317       0.0
            ATL                25             123.0     2.559394       1.0
            BAL                21              94.0    -0.093163       1.0
            BUF                31             197.0     4.315138       1.0
            CAR                33             113.0    -6.247634       0.0

In [8]:
# Get passing stats

# Total pass attempts, total pass yards, passing epa

passing = offense[offense.play_type == 'pass']
passing_grouped = passing.groupby(by=['season', 'week', 'posteam', 'passer_player_name'])
passing_grouped_no_sacks = passing[passing.sack == 0].groupby(by=['season', 'week', 'posteam', 'passer_player_name'])
pass_df = passing_grouped_no_sacks.count()['play_id'].rename('total_pass_attempts').to_frame()
pass_df['total_passing_yards'] = passing_grouped_no_sacks.sum()['yards_gained']
passing_grouped_completions = passing[passing.complete_pass == 1].groupby(by=['season', 'week', 'posteam', 'passer_player_name'])
pass_df[['completions', 'passing_epa', 'pass_tds']] = passing_grouped.sum()[['complete_pass', 'epa', 'pass_touchdown']]
pass_df[['air_yards', 'yards_after_catch', 'air_epa', 'yac_epa']] = passing_grouped_completions.sum()[['air_yards', 'yards_after_catch', 'air_epa', 'yac_epa']]
pass_df['avg_cpoe'] = passing_grouped_no_sacks.mean()['cpoe'].rename('avg_cpoe').to_frame()
pass_df = pass_df.reset_index(level='passer_player_name').rename(columns = {'passer_player_name':'qb'})

pass_df.tail(5)

qb  total_pass_attempts  total_passing_yards  \
season week posteam                                                          
2022   10   TEN      R.Tannehill                   36                255.0   
            WAS       T.Heinicke                   27                211.0   
       11   GB         A.Rodgers                   40                229.0   
            TEN          D.Henry                    2                  4.0   
            TEN      R.Tannehill                   27                333.0   

                     completions  passing_epa  pass_tds  air_yards  \
season week posteam                                                  
2022   10   TEN             19.0     4.792480       2.0      127.0   
            WAS             17.0     2.815080       0.0      141.0   
       11   GB              24.0     2.710083       2.0      137.0   
            TEN              2.0     0.466457       1.0        3.0   
            TEN             22.0    16.866169       2.0      190.0   

                     yards_after_catch    air_epa    yac_epa   avg_cpoe  
season week posteam                                                      
2022   10   TEN                  128.0   6.370534  16.488266 -15.691849  
            WAS                   70.0   8.408582   9.666575   6.855554  
       11   GB                    90.0   8.515738   8.720034  -0.817273  
            TEN                    1.0   0.896837  -0.430380  46.528168  
            TEN                  143.0  10.140337  19.858892  23.555149

In [29]:
# Get other offensive stats

# Get qb epa and rushing stats
qb_stats = offense[(offense.play_type == 'pass') | (offense.qb_scramble == 1) | (offense.qb_designed_run == 1)]
qb_stats['qb'] = qb_stats.apply(lambda x: x.passer_player_name if x.passer_player_name is not None else x.rusher_player_name, axis=1)

qb_epa_df = qb_stats.groupby(by = ['season', 'week', 'posteam', 'qb']).sum()['qb_epa'].rename('qb_epa').to_frame()

qb_runs = offense[((offense.qb_scramble == 1) & (offense.pass_length.isnull())) | (offense.qb_designed_run == 1)]
qb_runs['qb'] = qb_runs.apply(lambda x: x.passer_player_name if x.passer_player_name is not None else x.rusher_player_name, axis=1)

qb_rush_grouped = qb_runs.groupby(by=['season', 'week', 'posteam', 'qb'])
qb_rush_df = qb_rush_grouped.sum()[['yards_gained', 'epa', 'rush_touchdown']]
qb_rush_df.columns = ['qb_rush_yards', 'qb_rushing_epa', 'qb_rush_tds']

qb_carries = qb_rush_grouped.count()['epa'].rename('total_qb_rush_attempts')

qb_turnovers = qb_stats.groupby(by=['season', 'week', 'posteam', 'qb']).sum()[['sack', 'qb_hit', 'fumble', 'fumble_lost', 'interception']]
qb_turnovers.columns = ['sacks_taken_qb', 'qb_hits_taken_qb', 'fumbles_qb', 'lost_fumbles_qb', 'interceptions_thrown_qb']

qb_final = qb_epa_df.merge(qb_rush_df, left_index=True, right_index=True, how='left')
qb_final = qb_final.merge(qb_carries, left_index=True, right_index=True, how='left')
qb_final = qb_final.merge(qb_turnovers, left_index=True, right_index=True, how='left')
qb_final = qb_final.fillna(0)
# qb_final = qb_final.reset_index(level='qb')
qb_final.tail(5)

<ipython-input-29-2f7958a23ee1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_stats['qb'] = qb_stats.apply(lambda x: x.passer_player_name if x.passer_player_name is not None else x.rusher_player_name, axis=1)
<ipython-input-29-2f7958a23ee1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_runs['qb'] = qb_runs.apply(lambda x: x.passer_player_name if x.passer_player_name is not None else x.rusher_player_name, axis=1)


qb_epa  qb_rush_yards  qb_rushing_epa  \
season week posteam qb                                                      
2022   10   TEN     R.Tannehill   6.116046           12.0        1.323566   
            WAS     T.Heinicke    3.744092           14.0        0.929012   
       11   GB      A.Rodgers     2.710083            0.0        0.000000   
            TEN     D.Henry       0.466457            0.0        0.000000   
                    R.Tannehill  16.866169            0.0        0.000000   

                                 qb_rush_tds  total_qb_rush_attempts  \
season week posteam qb                                                 
2022   10   TEN     R.Tannehill          0.0                     2.0   
            WAS     T.Heinicke           0.0                     3.0   
       11   GB      A.Rodgers            0.0                     0.0   
            TEN     D.Henry              0.0                     0.0   
                    R.Tannehill          0.0                     0.0   

                                 sacks_taken_qb  qb_hits_taken_qb  fumbles_qb  \
season week posteam qb                                                          
2022   10   TEN     R.Tannehill             1.0               4.0         0.0   
            WAS     T.Heinicke              3.0               4.0         2.0   
       11   GB      A.Rodgers               1.0               3.0         0.0   
            TEN     D.Henry                 0.0               0.0         0.0   
                    R.Tannehill             3.0               4.0         0.0   

                                 lost_fumbles_qb  interceptions_thrown_qb  
season week posteam qb                                                     
2022   10   TEN     R.Tannehill              0.0                      0.0  
            WAS     T.Heinicke               1.0                      1.0  
       11   GB      A.Rodgers                0.0                      0.0  
            TEN     D.Henry                  0.0                      0.0  
                    R.Tannehill              0.0                      1.0

In [10]:
# Get sacks, interceptions, fumbles, fumbles lost, turnovers

turnovers_df = offense.groupby(by = ['season', 'week', 'posteam']).sum()[['sack', 'qb_hit', 'fumble', 'fumble_lost', 'interception']]
turnovers_df.columns = ['sacks_allowed_team', 'qb_hits_allowed_team', 'fumbles_team', 'lost_fumbles_team', 'interceptions_thrown_team']
turnovers_df.head(5)

sacks_allowed_team  qb_hits_allowed_team  fumbles_team  \
season week posteam                                                           
2014   1    ARI                     2.0                   4.0           2.0   
            ATL                     1.0                   4.0           1.0   
            BAL                     3.0                   6.0           1.0   
            BUF                     1.0                   3.0           0.0   
            CAR                     1.0                   3.0           1.0   

                     lost_fumbles_team  interceptions_thrown_team  
season week posteam                                                
2014   1    ARI                    2.0                        0.0  
            ATL                    1.0                        0.0  
            BAL                    1.0                        1.0  
            BUF                    0.0                        1.0  
            CAR                    0.0                        0.0

In [11]:
# offense[(offense.season==2021) & (offense.week==21) & (offense.posteam=='LA') & (offense.qb_scramble==1)]

In [12]:
# qb_runs = offense[((offense.qb_scramble == 1))]
# print(qb_runs[(qb_runs.season==2021) & (qb_runs.week==21) & (qb_runs.posteam=='LA')].pass_length.isnull())

In [13]:
# Get important defensive NFL stats per game per team

# Get only defensive plays, take out qb kneels

defense = nfl_df[nfl_df.play_type.isin(['run', 'pass'])]
defense = defense[defense.qb_kneel == 0]
defense.head(5)

play_id          game_id old_game_id home_team away_team season_type  week  \
2     51.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
3     72.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
4     93.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
6    141.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   
7    162.0  2014_01_BUF_CHI  2014090702       CHI       BUF         REG     1   

  posteam posteam_type defteam side_of_field  yardline_100   game_date  \
2     BUF         away     CHI           BUF          80.0  2014-09-07   
3     BUF         away     CHI           BUF          78.0  2014-09-07   
4     BUF         away     CHI           BUF          76.0  2014-09-07   
6     CHI         home     BUF           CHI          66.0  2014-09-07   
7     CHI         home     BUF           CHI          64.0  2014-09-07   

   quarter_seconds_remaining  half_seconds_remaining  game_seconds_remaining  \
2                      900.0                  1800.0                  3600.0   
3                      867.0                  1767.0                  3567.0   
4                      823.0                  1723.0                  3523.0   
6                      810.0                  1710.0                  3510.0   
7                      768.0                  1668.0                  3468.0   

  game_half  quarter_end  drive   sp  qtr  down  goal_to_go   time   yrdln  \
2     Half1          0.0    1.0  0.0  1.0   1.0         0.0  15:00  BUF 20   
3     Half1          0.0    1.0  0.0  1.0   2.0         0.0  14:27  BUF 22   
4     Half1          0.0    1.0  0.0  1.0   3.0         0.0  13:43  BUF 24   
6     Half1          0.0    2.0  0.0  1.0   1.0         0.0  13:30  CHI 34   
7     Half1          0.0    2.0  0.0  1.0   2.0         0.0  12:48  CHI 36   

   ydstogo  ydsnet                                               desc  \
2     10.0     4.0  (15:00) 28-C.Spiller left end to BUF 22 for 2 ...   
3      8.0     4.0  (14:27) (Shotgun) 28-C.Spiller left tackle to ...   
4      6.0     4.0  (13:43) (Shotgun) 3-E.Manuel pass incomplete s...   
6     10.0    66.0  (13:30) (Shotgun) 22-M.Forte right guard to CH...   
7      8.0    66.0  (12:48) (Shotgun) 6-J.Cutler pass short right ...   

  play_type  yards_gained  shotgun  no_huddle  qb_dropback  qb_kneel  \
2       run           2.0      0.0        0.0          0.0       0.0   
3       run           2.0      1.0        0.0          0.0       0.0   
4      pass           0.0      1.0        0.0          1.0       0.0   
6       run           2.0      1.0        0.0          0.0       0.0   
7      pass           8.0      1.0        0.0          1.0       0.0   

   qb_spike  qb_scramble pass_length pass_location  air_yards  \
2       0.0          0.0        None          None        NaN   
3       0.0          0.0        None          None        NaN   
4       0.0          0.0       short        middle       11.0   
6       0.0          0.0        None          None        NaN   
7       0.0          0.0       short         right        4.0   

   yards_after_catch run_location run_gap field_goal_result  kick_distance  \
2                NaN         left     end              None            NaN   
3                NaN         left  tackle              None            NaN   
4                NaN         None    None              None            NaN   
6                NaN        right   guard              None            NaN   
7                4.0         None    None              None            NaN   

  extra_point_result two_point_conv_result  home_timeouts_remaining  \
2               None                  None                      3.0   
3               None                  None                      3.0   
4               None                  None                      3.0   
6               None                  None                      3.0   
7               None              

In [14]:
# Get rushing defense stats

# Total rushes, total rush yards, rushing epa

rushes_def = defense[defense.play_type == 'run']
rushing_grouped_def = rushes.groupby(by=['season', 'week', 'defteam'])
rush_df_def = rushing_grouped_def.count()['play_id'].rename('total_rushes_allowed').to_frame()
rush_df_def[['total_rush_yards_allowed', 'rushing_epa_allowed', 'rush_tds_allowed']] = rushing_grouped_def.sum()[['yards_gained', 'epa', 'rush_touchdown']]

rush_df_def.head(5)

total_rushes_allowed  total_rush_yards_allowed  \
season week defteam                                                   
2014   1    ARI                        24                      52.0   
            ATL                        28                     139.0   
            BAL                        24                      81.0   
            BUF                        18                      86.0   
            CAR                        17                     102.0   

                     rushing_epa_allowed  rush_tds_allowed  
season week defteam                                         
2014   1    ARI                -8.805075               1.0  
            ATL                 4.264254               3.0  
            BAL                -4.888501               0.0  
            BUF                -0.055888               0.0  
            CAR                 1.025410               0.0

In [15]:
# Get passing defense stats

# Total pass attempts, total pass yards, passing epa

passing_def = defense[defense.play_type == 'pass']
passing_grouped_def = passing_def.groupby(by=['season', 'week', 'defteam'])
passing_grouped_def_no_sacks = passing_def[passing_def.sack == 0].groupby(by=['season', 'week', 'defteam'])
pass_df_def = passing_grouped_def_no_sacks.count()['play_id'].rename('total_pass_attempts_allowed').to_frame()
pass_df_def['passing_yards_allowed'] = passing_grouped_def_no_sacks.sum()['yards_gained']
passing_grouped_completions_def = passing_def[passing_def.complete_pass == 1].groupby(by=['season', 'week', 'defteam'])
pass_df_def[['completions_allowed', 'passing_epa_allowed', 'pass_tds_allowed']] = passing_grouped_def.sum()[['complete_pass', 'epa', 'pass_touchdown']]
pass_df_def[['air_yards_allowed', 'yards_after_catch_allowed', 'air_epa_allowed', 'yac_epa_allowed']] = passing_grouped_completions_def.sum()[['air_yards', 'yards_after_catch', 'air_epa', 'yac_epa']]
pass_df_def['avg_cpoe_allowed'] = passing_grouped_def_no_sacks.mean()['cpoe'].rename('avg_cpoe').to_frame()

pass_df_def.tail(5)

total_pass_attempts_allowed  passing_yards_allowed  \
season week defteam                                                       
2022   10   TB                                34                  275.0   
            TEN                               42                  286.0   
            WAS                               26                  175.0   
       11   GB                                29                  337.0   
            TEN                               40                  229.0   

                     completions_allowed  passing_epa_allowed  \
season week defteam                                             
2022   10   TB                      23.0             3.934149   
            TEN                     21.0            -6.271894   
            WAS                     17.0           -10.139235   
       11   GB                      24.0            17.332626   
            TEN                     24.0             2.710083   

                     pass_tds_allowed  air_yards_allowed  \
season week defteam                                        
2022   10   TB                    2.0              149.0   
            TEN                   1.0              109.0   
            WAS                   2.0              124.0   
       11   GB                    3.0              193.0   
            TEN                   2.0              137.0   

                     yards_after_catch_allowed  air_epa_allowed  \
season week defteam                                               
2022   10   TB                           126.0         8.474874   
            TEN                          177.0         1.285257   
            WAS                           65.0         5.515973   
       11   GB                           144.0        11.037174   
            TEN                           90.0         8.515738   

                     yac_epa_allowed  avg_cpoe_allowed  
season week defteam                                     
2022   10   TB             14.029530          5.180484  
            TEN            19.344631         -9.952860  
            WAS            -6.568710         -0.824411  
       11   GB             19.428513         25.256855  
            TEN             8.720034         -0.817273

In [16]:
# Get other defensive stats

# Get defensive qb epa and rushing stats
qb_stats_def = defense[(defense.play_type == 'pass') | (defense.qb_scramble == 1) | (defense.qb_designed_run == 1)]
qb_epa_df_def = qb_stats_def.groupby(by = ['season', 'week', 'defteam']).sum()['qb_epa'].rename('qb_epa_allowed').to_frame()

qb_runs_def = defense[((defense.qb_scramble == 1) & (defense.pass_length.isnull())) | (defense.qb_designed_run == 1)]
qb_rush_grouped_def = qb_runs_def.groupby(by=['season', 'week', 'defteam'])
qb_rush_df_def = qb_rush_grouped_def.sum()[['yards_gained', 'epa', 'rush_touchdown']]
qb_rush_df_def.columns = ['qb_rush_yards_allowed', 'qb_rushing_epa_allowed', 'qb_rush_tds_allowed']

qb_final_def = qb_epa_df_def.merge(qb_rush_df_def, left_index=True, right_index=True, how='left')
qb_final_def = qb_final_def.fillna(0)
qb_final_def.tail(5)

qb_epa_allowed  qb_rush_yards_allowed  \
season week defteam                                          
2022   10   TB             5.720709                   22.0   
            TEN           -6.894927                    9.0   
            WAS            8.824848                   28.0   
       11   GB            17.332626                    0.0   
            TEN            2.710083                    0.0   

                     qb_rushing_epa_allowed  qb_rush_tds_allowed  
season week defteam                                               
2022   10   TB                     1.786560                  0.0  
            TEN                   -0.623033                  0.0  
            WAS                    2.287845                  1.0  
       11   GB                     0.000000                  0.0  
            TEN                    0.000000                  0.0

In [17]:
# Get sacks, interceptions, fumbles, fumbles lost, turnovers

turnovers_df_def = defense.groupby(by = ['season', 'week', 'defteam']).sum()[['sack', 'qb_hit', 'fumble_forced', 'interception']]
turnovers_df_def.columns = ['sacks', 'qb_hits', 'fumbles_forced', 'interceptions']
fumble_recovery_df = defense[(defense.defteam == defense.fumble_recovery_1_team)].groupby(by = ['season', 'week', 'defteam']).sum()['fumble_forced'].rename('fumbles_recovered').to_frame()
turnovers_df_def = turnovers_df_def.join(fumble_recovery_df, how='left').fillna(0)
turnovers_df_def.head(5)

sacks  qb_hits  fumbles_forced  interceptions  \
season week defteam                                                  
2014   1    ARI        0.0      4.0             0.0            1.0   
            ATL        0.0      0.0             1.0            1.0   
            BAL        0.0      1.0             0.0            0.0   
            BUF        2.0      4.0             1.0            2.0   
            CAR        3.0      9.0             2.0            2.0   

                     fumbles_recovered  
season week defteam                     
2014   1    ARI                    0.0  
            ATL                    1.0  
            BAL                    0.0  
            BUF                    1.0  
            CAR                    1.0

In [18]:
# Get special teams stats

specials = nfl_df[nfl_df.special == 1]
specials_grouped = specials.groupby(by = ['season', 'week', 'posteam'])
specials_epa_df_one = specials_grouped.sum()['epa'].rename('special_teams_epa_one').to_frame()

specials = nfl_df[nfl_df.special == 1]
specials_grouped = specials.groupby(by = ['season', 'week', 'defteam'])
specials_epa_df_two = specials_grouped.sum()['epa'].rename('special_teams_epa_two').to_frame()

specials_epa_df = specials_epa_df_one.merge(specials_epa_df_two, left_on=['season', 'week', 'posteam'], right_index=True)
specials_epa_df['special_teams_epa'] = specials_epa_df.special_teams_epa_one - specials_epa_df.special_teams_epa_two
specials_epa_df.drop(columns = ['special_teams_epa_one', 'special_teams_epa_two'], inplace=True)
specials_epa_df.head(5)

special_teams_epa
season week posteam                   
2014   1    ARI              -4.751772
            ATL              -1.162613
            BAL              -2.107624
            BUF               3.721131
            CAR              -3.457676

In [19]:
# Get overall score and other game total stats

home_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'home_team', 'away_team']).max()[['home_score', 'away_score']]
home_scores_and_etc_df.index.names = ['season', 'week', 'team', 'opponent']
home_scores_and_etc_df.columns = ['score', 'opponent_score']
away_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'away_team', 'home_team']).max()[['away_score', 'home_score']]
away_scores_and_etc_df.index.names = ['season', 'week', 'team', 'opponent']
away_scores_and_etc_df.columns = ['score', 'opponent_score']

final_nfl_df = pd.concat([home_scores_and_etc_df, away_scores_and_etc_df])
final_nfl_df = final_nfl_df.sort_index()
final_nfl_df.head(5)

<ipython-input-19-84b6000b41ce>:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  home_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'home_team', 'away_team']).max()[['home_score', 'away_score']]
<ipython-input-19-84b6000b41ce>:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  away_scores_and_etc_df = nfl_df.groupby(by = ['season', 'week', 'away_team', 'home_team']).max()[['away_score', 'home_score']]


score  opponent_score
season week team opponent                       
2014   1    ARI  LAC          18              17
            ATL  NO           37              34
            BAL  CIN          16              23
            BUF  CHI          23              20
            CAR  TB           20              14

In [20]:
# Combine all stats together into one data frame

final_nfl_df = final_nfl_df.merge(
    pass_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    rush_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    qb_final, left_on=['season', 'week', 'team', 'qb'], right_index=True).merge(
    turnovers_df, left_on=['season', 'week', 'team'], right_index=True).merge(
    rush_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    pass_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    qb_final_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    turnovers_df_def, left_on=['season', 'week', 'team'], right_index=True).merge(
    specials_epa_df, left_on=['season', 'week', 'team'], right_index=True)

final_nfl_df.head(5)

score  opponent_score          qb  \
season week team opponent                                      
2014   1    ARI  LAC          18              17    C.Palmer   
            ATL  NO           37              34      M.Ryan   
            BAL  CIN          16              23    J.Flacco   
            BUF  CHI          23              20    E.Manuel   
            CAR  TB           20              14  D.Anderson   

                           total_pass_attempts  total_passing_yards  \
season week team opponent                                             
2014   1    ARI  LAC                        38                304.0   
            ATL  NO                         42                448.0   
            BAL  CIN                        62                345.0   
            BUF  CHI                        22                173.0   
            CAR  TB                         33                230.0   

                           completions  passing_epa  pass_tds  air_yards  \
season week team opponent                                                  
2014   1    ARI  LAC              24.0     6.098425       2.0      196.0   
            ATL  NO               31.0    17.253965       3.0      253.0   
            BAL  CIN              35.0    -4.871387       1.0      221.0   
            BUF  CHI              16.0     0.368607       1.0       93.0   
            CAR  TB               24.0     8.423189       2.0      160.0   

                           yards_after_catch    air_epa    yac_epa   avg_cpoe  \
season week team opponent                                                       
2014   1    ARI  LAC                   108.0  10.800692  13.884696   3.160065   
            ATL  NO                    195.0   8.815695  16.383532   6.128030   
            BAL  CIN                   124.0  10.962930  14.313325 -11.889451   
            BUF  CHI                    80.0   1.909633   9.530254  11.150598   
            CAR  TB                     70.0   6.298791  10.724494   9.822589   

                           total_rushes  total_rush_yards  rushing_epa  \
season week team opponent                                                
2014   1    ARI  LAC                 26             110.0    -7.697317   
            ATL  NO                  25             123.0     2.559394   
            BAL  CIN                 21              94.0    -0.093163   
            BUF  CHI                 31             197.0     4.315138   
            CAR  TB                  33             113.0    -6.247634   

                           rush_tds     qb_epa  qb_rush_yards  qb_rushing_epa  \
season week team opponent                                                       
2014   1    ARI  LAC            0.0   7.779734           30.0        1.681309   
            ATL  NO             1.0  24.116819           15.0        0.272050   
            BAL  CIN            1.0  -3.078329            7.0        1.793058   
            BUF  CHI            1.0   0.591515           23.0        0.222908   
            CAR  TB             0.0  10.352312           10.0        1.929123   

                           qb_rush_tds  sacks_taken_qb  qb_hits_taken_qb  \
season week team opponent                                                  
2014   1    ARI  LAC               0.0             2.0               4.0   
            ATL  NO                0.0             1.0               4.0   
            BAL  CIN               0.0             3.0               6.0   
            BUF  CHI               1.0             1.0               3.0   
            CAR  TB                0.0             1.0               3.0   

                           fumbles_qb  lost_fumbles_qb  \
season week team opponent                                
2014   1    ARI  LAC              1.0              1.0   
            ATL  NO               1.0              1.0   
            BAL  CIN              0.0              0.0   
            BUF  CHI              0.0              0.0   
           

In [21]:
# Flip sign of def epa 

# final_nfl_df['passing_epa_def'] = -final_nfl_df['passing_epa_allowed']
# final_nfl_df['rushing_epa_def'] = -final_nfl_df['rushing_epa_allowed']

# final_nfl_df['qb_epa_def'] = -final_nfl_df['qb_epa_allowed']
# final_nfl_df['qb_rushing_epa_def'] = -final_nfl_df['qb_rushing_epa_allowed']

# final_nfl_df['air_epa_def'] = -final_nfl_df['air_epa_allowed']
# final_nfl_df['yac_epa_def'] = -final_nfl_df['yac_epa_allowed']

# Add total epa stats

final_nfl_df['total_epa'] = final_nfl_df.apply(lambda x: x.passing_epa + x.rushing_epa + x.special_teams_epa - x.rushing_epa_allowed - x.passing_epa_allowed, axis = 1)
final_nfl_df['total_opposing_epa'] = -final_nfl_df.total_epa

In [22]:
# Fix QB Names

def get_abbreviated_qb(qb):
    qb_split = qb.split('.')
    first = qb_split[0][0] + '.'
    last = qb_split[-1]
    qb_full = first + ' ' + last
    return qb_full

final_nfl_df_fixed_qb = final_nfl_df.copy()
final_nfl_df_fixed_qb['qb'] = final_nfl_df_fixed_qb.apply(lambda x: get_abbreviated_qb(x.qb), axis=1)
final_nfl_df_fixed_qb

score  opponent_score            qb  \
season week team opponent                                        
2014   1    ARI  LAC          18              17     C. Palmer   
            ATL  NO           37              34       M. Ryan   
            BAL  CIN          16              23     J. Flacco   
            BUF  CHI          23              20     E. Manuel   
            CAR  TB           20              14   D. Anderson   
...                          ...             ...           ...   
2022   10   TEN  DEN          17              10  R. Tannehill   
            WAS  PHI          32              21   T. Heinicke   
       11   GB   TEN          17              27    A. Rodgers   
            TEN  GB           27              17      D. Henry   
                 GB           27              17  R. Tannehill   

                           total_pass_attempts  total_passing_yards  \
season week team opponent                                             
2014   1    ARI  LAC                        38                304.0   
            ATL  NO                         42                448.0   
            BAL  CIN                        62                345.0   
            BUF  CHI                        22                173.0   
            CAR  TB                         33                230.0   
...                                        ...                  ...   
2022   10   TEN  DEN                        36                255.0   
            WAS  PHI                        27                211.0   
       11   GB   TEN                        40                229.0   
            TEN  GB                          2                  4.0   
                 GB                         27                333.0   

                           completions  passing_epa  pass_tds  air_yards  \
season week team opponent                                                  
2014   1    ARI  LAC              24.0     6.098425       2.0      196.0   
            ATL  NO               31.0    17.253965       3.0      253.0   
            BAL  CIN              35.0    -4.871387       1.0      221.0   
            BUF  CHI              16.0     0.368607       1.0       93.0   
            CAR  TB               24.0     8.423189       2.0      160.0   
...                                ...          ...       ...        ...   
2022   10   TEN  DEN              19.0     4.792480       2.0      127.0   
            WAS  PHI              17.0     2.815080       0.0      141.0   
       11   GB   TEN              24.0     2.710083       2.0      137.0   
            TEN  GB                2.0     0.466457       1.0        3.0   
                 GB               22.0    16.866169       2.0      190.0   

                           yards_after_catch    air_epa    yac_epa   avg_cpoe  \
season week team opponent                                                       
2014   1    ARI  LAC                   108.0  10.800692  13.884696   3.160065   
            ATL  NO                    195.0   8.815695  16.383532   6.128030   
            BAL  CIN                   124.0  10.962930  14.313325 -11.889451   
            BUF  CHI                    80.0   1.909633   9.530254  11.150598   
            CAR  TB                     70.0   6.298791  10.724494   9.822589   
...                                      ...        ...        ...        ...   
2022   10   TEN  DEN                   128.0   6.370534  16.488266 -15.691849   
            WAS  PHI                    70.0   8.408582   9.666575   6.855554   
       11   GB   TEN                    90.0   8.515738   8.720034  -0.817273   
            TEN  GB                      1.0   0.896837  -0.430380  46.528168   
                 GB                    143.0  10.140337  19.858892  23.555149   

                           total_rushes  total_rush_yards  rushing_epa  \
season week team opponent                                                
2014   1    ARI  LAC                 26             1

In [23]:
final_nfl_df_fixed_qb.tail()

score  opponent_score            qb  \
season week team opponent                                        
2022   10   TEN  DEN          17              10  R. Tannehill   
            WAS  PHI          32              21   T. Heinicke   
       11   GB   TEN          17              27    A. Rodgers   
            TEN  GB           27              17      D. Henry   
                 GB           27              17  R. Tannehill   

                           total_pass_attempts  total_passing_yards  \
season week team opponent                                             
2022   10   TEN  DEN                        36                255.0   
            WAS  PHI                        27                211.0   
       11   GB   TEN                        40                229.0   
            TEN  GB                          2                  4.0   
                 GB                         27                333.0   

                           completions  passing_epa  pass_tds  air_yards  \
season week team opponent                                                  
2022   10   TEN  DEN              19.0     4.792480       2.0      127.0   
            WAS  PHI              17.0     2.815080       0.0      141.0   
       11   GB   TEN              24.0     2.710083       2.0      137.0   
            TEN  GB                2.0     0.466457       1.0        3.0   
                 GB               22.0    16.866169       2.0      190.0   

                           yards_after_catch    air_epa    yac_epa   avg_cpoe  \
season week team opponent                                                       
2022   10   TEN  DEN                   128.0   6.370534  16.488266 -15.691849   
            WAS  PHI                    70.0   8.408582   9.666575   6.855554   
       11   GB   TEN                    90.0   8.515738   8.720034  -0.817273   
            TEN  GB                      1.0   0.896837  -0.430380  46.528168   
                 GB                    143.0  10.140337  19.858892  23.555149   

                           total_rushes  total_rush_yards  rushing_epa  \
season week team opponent                                                
2022   10   TEN  DEN                 23              63.0    -9.867366   
            WAS  PHI                 47             156.0     3.071281   
       11   GB   TEN                 19              56.0    -4.261430   
            TEN  GB                  29              91.0    -2.641367   
                 GB                  29              91.0    -2.641367   

                           rush_tds     qb_epa  qb_rush_yards  qb_rushing_epa  \
season week team opponent                                                       
2022   10   TEN  DEN            0.0   6.116046           12.0        1.323566   
            WAS  PHI            2.0   3.744092           14.0        0.929012   
       11   GB   TEN            0.0   2.710083            0.0        0.000000   
            TEN  GB             1.0   0.466457            0.0        0.000000   
                 GB             1.0  16.866169            0.0        0.000000   

                           qb_rush_tds  sacks_taken_qb  qb_hits_taken_qb  \
season week team opponent                                                  
2022   10   TEN  DEN               0.0             1.0               4.0   
            WAS  PHI               0.0             3.0               4.0   
       11   GB   TEN               0.0             1.0               3.0   
            TEN  GB                0.0             0.0               0.0   
                 GB                0.0             3.0               4.0   

                           fumbles_qb  lost_fumbles_qb  \
season week team opponent                                
2022   10   TEN  DEN              0.0              0.0   
            WAS  PHI              2.0              1.0   
       11   GB   TEN              0.0              0.0   
            TEN  GB               0.0              0.0 

In [24]:
final_nfl_df_fixed_qb.to_csv('../data/nfl_game_by_game_raw_stats.csv')

In [25]:
# Get game schedule and results

schedule_df = final_nfl_df.copy()
schedule_df = schedule_df[['score', 'opponent_score']].reset_index()
schedule_df.head()

season  week team opponent  score  opponent_score
0    2014     1  ARI      LAC     18              17
1    2014     1  ATL       NO     37              34
2    2014     1  BAL      CIN     16              23
3    2014     1  BUF      CHI     23              20
4    2014     1  CAR       TB     20              14

In [26]:
schedule_df.to_csv('../data/schedule_df.csv')